<a href="https://colab.research.google.com/github/dapopov-st/ExperimentsWithLanguageModels/blob/main/RAG_over_ArXiv_PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Make a directory of ArXiv papers on Google Drive
- See if can store the vector store in Drive or just in ram
- Bridge this paper with the book
- If can't access A100 consistenly and run out of GPU memory, make a Mistral 7B/Zephyr 7B version (Zephyr can do ReAct!)
- Rag with metadata (ex, subdirectory name etc.)
- Use Self-Rag 7b? selfrag/selfrag_llama2_7b or 13b.  Think this will help overcome some of the challenges with retrieving from pdfs (getting titles from References section, for example, rather than useful content)

- Original NB worked in under 10GB on V100

### Pre-task Work

All we really need to do to get started is to get our prerequisites!

We'll be leveraging `langchain` and `llama 2` today.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [LLaMA 2](https://huggingface.co/blog/llama2)

In [1]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"
!pip install -U -q cohere llama-index
!pip install PyPDF2
!pip install pypdf
!pip install -q qdrant-client
!pip install -q -U faiss-cpu tiktoken sentence-transformers
import os
from google.colab import drive
drive.mount('/content/drive/')

output_dir = '/content/drive/MyDrive/PdfRag/rag_output_dir'
logging_dir = '/content/drive/MyDrive/PdfRag/rag_logging_dir'
index_dir = '/content/drive/MyDrive/PdfRag/rag_index_dir'

#!ls /content/drive/MyDrive/PdfRag/clusterofstars
%cd /content/drive/MyDrive/PdfRag
#My\ Drive/PdfRag && ls clusterofstars

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/PdfRag


In [2]:
!ls .

barbie.csv  cache  clusterofstars  oppenheimer.csv  rag_index_dir


In [3]:
from pathlib import Path
PDFS_PATH = Path('/content/drive/MyDrive/PdfRag/clusterofstars')
PDFS = list(PDFS_PATH.glob('*.pdf'))
PDFS[0], len(PDFS)

(PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/InContextRALM.pdf'),
 28)

In [4]:
# For now, flatten out the file structure for simplicity.  In the future, see if can have Rag use this metadata.
#!mv /content/drive/MyDrive/PdfRag/clusterofstars/ChainOfThought/* /content/drive/MyDrive/PdfRag/clusterofstars

In [5]:
# fastai function to clean GPU memory

import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()



### Build a PDF Data Loader

In [6]:
!pip install --q pdfminer.six

In [7]:
from langchain.document_loaders import PDFMinerLoader

In [8]:
loader = PDFMinerLoader(os.path.expanduser(PDFS[0]))

In [9]:
pdf_content = loader.load()
pdf_content

[Document(page_content='In-ContextRetrieval-AugmentedLanguageModelsOriRam∗YoavLevine∗ItayDalmedigosDorMuhlgayAmnonShashuaKevinLeyton-BrownYoavShohamAI21Labs{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.comAbstractRetrieval-AugmentedLanguageModeling(RALM)methods,whichconditionalan-guagemodel(LM)onrelevantdocumentsfromagroundingcorpusduringgeneration,wereshowntosigni\ue000cantlyimprovelan-guagemodelingperformance.Inaddition,theycanmitigatetheproblemoffactuallyinaccuratetextgenerationandprovidenatu-ralsourceattributionmechanism.ExistingRALMapproachesfocusonmodifyingtheLMarchitectureinordertofacilitatethein-corporationofexternalinformation,signi\ue000-cantlycomplicatingdeployment.Thispaperconsidersasimplealternative,whichwedubIn-ContextRALM:leavingtheLMarchitec-tureunchangedandprependinggroundingdocumentstotheinput,withoutanyfurthertrainingoftheLM.WeshowthatIn-ContextRALMthatbuildsonoff-the-shelfgeneralpurposeretrieversprovidessurprisinglylargeLMgainsacrossmodelsizesanddiversecor-pora.

In [36]:
from PyPDF2 import PdfReader
reader = PdfReader(os.path.expanduser(PDFS[0]))
pages = reader.pages
documents = []
for page in pages:
  documents.append(page.extract_text())
print(documents[-1])

(tokens) is optimal. This is in contrast to similar
experiments we conducted for BM25 (cf. Figure 6),
whereℓ= 32is optimal.
B GPT-Neo Results
Table 5gives the results of applying In-Context
RALM to the models from the GPT-Neo model
family on WikiText-103 and RealNews.
C Open-Domain Question Answering
Experiments: Further Details
Closed-Book SettingFor the closed-book set-
ting, we adopt the prompt of Touvron et al. (2023 ):
Answer these questions:
Q: Who got the first nobel
prize in physics?
A:
Open-Book SettingFor the open-book setting,
we extend the above prompt as follows:
Nobel Prize
A group including 42
Swedish writers, artists,
and literary critics
protested against this
decision, having expected
Leo Tolstoy to be awarded.
Some, including Burton
Feldman, have criticised
this prize because they...
Nobel Prize in Physiology
or Medicine
In the last half century
there has been an
increasing tendency
for scientists to work
as teams, resulting in
controversial exclusions.
Alfred Nobel 

## First drop everything from References onwards. References were 'confusing' RAG into retrieving primarily titles of papers mentioned there, which is likely not very useful

In [140]:
import PyPDF2

def load_pdf_to_string(pdf_path):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to hold the text
        text = ''

        # Loop through each page and extract the text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            references_index= page_text.upper().find('\nREFERENCES\n')
            if references_index != -1:
              page_text = page_text[:references_index]
              text += page_text
              return text
            text += page_text
    return text

# Use the function to load a PDF into a string
text = load_pdf_to_string(os.path.expanduser(PDFS[1]))

In [118]:
text.find('References\n')

-1

In [90]:
PDFS[0]

PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/InContextRALM.pdf')

In [141]:
all_docs = [load_pdf_to_string(os.path.expanduser(pdf_path)) for pdf_path in PDFS]

In [120]:
len(all_docs)

28

In [121]:
all_docs[-1].find('References\n')

-1

In [34]:
# from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader(os.path.expanduser(PDFS[0]))
# #pages = loader.load_and_split()
# pages = loader.load()
# all_pages = [load_pdf_without_references(i) for i in range(len(PDFS))]

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


AttributeError: ignored

In [123]:
all_docs[0]

'In-Context Retrieval-Augmented Language Models\nOri Ram∗Yoav Levine∗Itay Dalmedigos Dor Muhlgay\nAmnon Shashua Kevin Leyton-Brown Yoav Shoham\nAI21 Labs\n{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.com\nAbstract\nRetrieval-Augmented Language Modeling\n(RALM) methods, which condition a lan-\nguage model (LM) on relevant documents\nfrom a grounding corpus during generation,\nwere shown to signi\ue000cantly improve lan-\nguage modeling performance. In addition,\nthey can mitigate the problem of factually\ninaccurate text generation and provide natu-\nral source attribution mechanism. Existing\nRALM approaches focus on modifying the\nLM architecture in order to facilitate the in-\ncorporation of external information, signi\ue000-\ncantly complicating deployment. This paper\nconsiders a simple alternative, which we dub\nIn-Context RALM: leaving the LM architec-\nture unchanged and prepending grounding\ndocuments to the input,without any further\ntraining of the LM. We show that In-Co

In [65]:
# from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader(os.path.expanduser(PDFS[0]))
# #pages = loader.load_and_split()
# pages = loader.load()
# all_pages = [PyPDFLoader(os.path.expanduser(PDFS[i])).load() for i in range(len(PDFS))]

- Instead of load_and_split, going for a more custom split to have more control over context length sicnce working with limited compute

In [66]:
#pages[0], len(pages)

-Either PyPDFLoader or PyPDF2 approach could work

In [67]:
# from langchain.document_loaders.onedrive_file import CHUNK_SIZE
# from langchain.document_loaders import TextLoader
# from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 30

# text_splitter = CharacterTextSplitter(
#     chunk_size=CHUNK_SIZE,
#     chunk_overlap = CHUNK_OVERLAP,
#     length_function=len,
# )
# docs  = text_splitter.split_documents(pages)
# docs

In [146]:
from langchain.document_loaders.onedrive_file import CHUNK_SIZE
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.text_splitter import Document

# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 30
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 30

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    length_function=len,
)
#text_splitter.split_text(all_pages[0])
# docs = [Document(page_content=pages) for pages in all_pages]
docs  = [text_splitter.split_text(doc) for doc in all_docs]
# # docs

In [125]:
len(docs)

28

In [127]:
docs[0]

['In-Context Retrieval-Augmented Language Models\nOri Ram∗Yoav Levine∗Itay Dalmedigos Dor Muhlgay\nAmnon Shashua Kevin Leyton-Brown Yoav Shoham\nAI21 Labs\n{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.com\nAbstract\nRetrieval-Augmented Language Modeling\n(RALM) methods, which condition a lan-\nguage model (LM) on relevant documents\nfrom a grounding corpus during generation,\nwere shown to signi\ue000cantly improve lan-\nguage modeling performance. In addition,\nthey can mitigate the problem of factually\ninaccurate text generation and provide natu-\nral source attribution mechanism. Existing\nRALM approaches focus on modifying the\nLM architecture in order to facilitate the in-\ncorporation of external information, signi\ue000-\ncantly complicating deployment. This paper\nconsiders a simple alternative, which we dub\nIn-Context RALM: leaving the LM architec-\nture unchanged and prepending grounding\ndocuments to the input,without any further\ntraining of the LM. We show that In-C

In [86]:
PDFS[-1]

PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Self-Rag.pdf')

In [16]:
# from langchain.document_loaders.onedrive_file import CHUNK_SIZE
# from langchain.document_loaders import TextLoader
# from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 30

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=CHUNK_SIZE,
#     chunk_overlap = CHUNK_OVERLAP,
#     length_function=len,
# )
# #docs  = text_splitter.split_documents([doc.page_content for doc in all_pages])
# docs = text_splitter.split_documents(all_pages)
len(docs)

28

In [17]:
len(docs[0])

62

- Not yet sure if should use Qdrant or FAISS


In [128]:
from langchain.vectorstores import Qdrant, FAISS

In [129]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents((docs[i][j] for i in range(len(docs)) for j in range(len(docs[i]))), embedder)

In [136]:
#docs = [Document(page_content=doc[0]) for doc in docs]

In [144]:
content

Document(page_content='In-Context Retrieval-Augmented Language Models\nOri Ram∗Yoav Levine∗Itay Dalmedigos Dor Muhlgay\nAmnon Shashua Kevin Leyton-Brown Yoav Shoham\nAI21 Labs\n{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.com\nAbstract\nRetrieval-Augmented Language Modeling\n(RALM) methods, which condition a lan-\nguage model (LM) on relevant documents\nfrom a grounding corpus during generation,\nwere shown to signi\ue000cantly improve lan-\nguage modeling performance. In addition,\nthey can mitigate the problem of factually\ninaccurate text generation and provide natu-\nral source attribution mechanism. Existing\nRALM approaches focus on modifying the\nLM architecture in order to facilitate the in-\ncorporation of external information, signi\ue000-\ncantly complicating deployment. This paper\nconsiders a simple alternative, which we dub\nIn-Context RALM: leaving the LM architec-\nture unchanged and prepending grounding\ndocuments to the input,without any further\ntraining of the 

In [147]:
#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents(docs[0], embedder)
from langchain.schema.document import Document

docs = [Document(page_content=doc[i]) for doc in docs for i in range(len(doc))]
for index, pdf in enumerate(docs):
   content = docs[index]
   if index == 0:
       vector_store = FAISS.from_documents([content], embedder)
   else:
      vector_store_i = FAISS.from_documents([content], embedder)
      vector_store.merge_from(vector_store_i)


vector_store.save_local(index_dir)

In [149]:
query = "What is Retrieval-augmented generation?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

1 I NTRODUCTION
State-of-the-art LLMs continue to struggle with factual errors (Mallen et al., 2023; Min et al., 2023)
despite their increased model and data scale (Ouyang et al., 2022). Retrieval-Augmented Generation
(RAG) methods (Figure 1 left; Lewis et al. 2020; Guu et al. 2020) augment the input of LLMs
with relevant retrieved passages, reducing factual errors in knowledge-intensive tasks (Ram et al.,
2023; Asai et al., 2023a). However, these methods may hinder the versatility of LLMs or introduce
unnecessary or off-topic passages that lead to low-quality generations (Shi et al., 2023) since they
retrieve passages indiscriminately regardless of whether the factual grounding is helpful. Moreover,
the output is not guaranteed to be consistent with retrieved relevant passages (Gao et al., 2023) since
the models are not explicitly trained to leverage and follow facts from provided passages. This
work introduces Self-Reflective Retrieval-augmented Generation ( SELF-RAG)to improve an
LL

In [150]:
query = "What is Self-Rag?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

predictions are mostly aligned with their assessments. Appendix Table 6 shows several annotated
examples and explanations on assessments.
6 C ONCLUSION
This work introduces SELF-RAG, a new framework to enhance the quality and factuality of LLMs
through retrieval on demand and self-reflection. SELF-RAGtrains an LM to learn to retrieve, generate,
and critique text passages and its own generation by predicting the next tokens from its original
vocabulary as well as newly added special tokens, called reflection tokens. SELF-RAGfurther enables
the tailoring of LM behaviors at test time by leveraging reflection tokens. Our holistic evaluations on
six tasks using multiple metrics demonstrate that SELF-RAGsignificantly outperforms LLMs with
more parameters or with conventional retrieval-augmented generation approaches.
10Preprint.
ETHICAL CONCERNS
This work aims to improve the factuality of LLM outputs, the lack of which continues to cause nu-
3 S ELF-RAG: LEARNING TO RETRIEVE , GENERATE AND C

## Looks like retrieving from References section may be a problem!  Could experiment with different retriever or a smart-enough model (Self-Rag?)

## LOOKS LIKE REMOVING REFERENCES ONWARDS INCREASED THE RETRIEVAL QUALITY NOTICEABLY

### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

In [63]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/oppenheimer.csv

--2023-11-04 20:56:55--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/oppenheimer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80957 (79K) [text/plain]
Saving to: ‘oppenheimer.csv’

oppenheimer.csv     100%[===================>]  79.06K  --.-KB/s    in 0.002s  

2023-11-04 20:56:55 (33.7 MB/s) - ‘oppenheimer.csv’ saved [80957/80957]



In [64]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/barbie.csv

--2023-11-04 20:56:57--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/barbie.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72289 (71K) [text/plain]
Saving to: ‘barbie.csv’

barbie.csv          100%[===================>]  70.59K  --.-KB/s    in 0.002s  

2023-11-04 20:56:57 (33.3 MB/s) - ‘barbie.csv’ saved [72289/72289]



#### Data Parsing

Now that we have our data - let's go ahead and start parsing it into a more usable format for LangChain!

We'll be using the `CSVLoader` for this application.

Check out the docs here:
- [CSVLoader](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [61]:
from langchain.document_loaders.csv_loader import CSVLoader

In [65]:
barbie_loader = CSVLoader(file_path="barbie.csv", source_column="Review_Url")

barbie_data = barbie_loader.load()

In [66]:
len(barbie_data)

125

In [67]:
oppenheimer_loader = CSVLoader(file_path="oppenheimer.csv", source_column="Review_Url")

oppenheimer_data = oppenheimer_loader.load()

In [68]:
len(oppenheimer_data)

150

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

In [70]:
barbie_documents = text_splitter.transform_documents(barbie_data)

In [71]:
len(barbie_documents)

181

In [72]:
oppenheimer_documents = text_splitter.transform_documents(oppenheimer_data)

In [73]:
len(oppenheimer_documents)

186

In [74]:
combined_documents = barbie_documents + oppenheimer_documents

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

### Task 2: Creating an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

#### Selecting Our VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [55]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper

We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [75]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(combined_documents, embedder)

Now that we've created the VectorStore, we can check that it's working by embedding a query and retrieving passages from our reviews that are close to it.

In [23]:
query = "How is Will Ferrell in this movie?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

Published as a conference paper at ICLR 2023
Fever Prompts – Continued from previous page
Claim Beautiful reached number two on the Billboard Hot 100 in 2003.
Thought The song peaked at number two on the Billboard Hot 100 in the United States,
but not sure if it was in 2003.
Answer NOT ENOUGH INFO
ReAct Determine if there is Observation that SUPPORTS or REFUTES a Claim, or if
there is NOT ENOUGH INFORMATION.
Claim Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
Thought 1 I need to search Nikolaj Coster-Waldau and find if he has worked with the
Fox Broadcasting Company.
Action 1 Search[Nikolaj Coster-Waldau]
Observation 1 Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and
producer. He graduated from the Danish National School of Performing Arts
in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with
the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy
Published as a conference paper at ICLR 2023
C.2 F EVER
FEVER Pr

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [ ]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

12.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

7.89 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we're going to leverage Meta's LLaMA 2!

Specifically, we'll be using: `meta-llama/Llama-2-13b-chat-hf`

That's right, a 13B parameter model that we're going to run on *less than* 15GB of GPU RAM.

More information on this model can be found [here](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)

In [ ]:
!pip install huggingface-hub -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

We will be leveraging Tim Dettmer's `bitsandbytes` as well as `accelerate` and `transformers` from Hugging Face to make our model as small as possible. The overall quality of the model is fairly well retained!

In [ ]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

(…)a-2-13b-chat-hf/resolve/main/config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-13b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [ ]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 4.0 MB/s eta 0:00:00


In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.0,
    max_new_tokens=256
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Now that it's set-up, let's test it out!

In [ ]:
qa_with_sources_chain({"query" : "How was Will Ferrell in this movie?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How was Will Ferrell in this movie?',
 'result': ' Based on the reviews, Will Ferrell\'s character was not well received by some of the reviewers. One reviewer described his character as "ruining every scene he was in." Another reviewer mentioned that his board became "superfluous." However, another reviewer found his performance to be enjoyable. Overall, it seems that opinions on Will Ferrell\'s performance in the movie are mixed.',
 'source_documents': [Document(page_content=": 61\nReview_Date: 23 July 2023\nAuthor: agjbull\nRating: 6\nReview_Title: Just a little empty\nReview: I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow. The film was very disjointed. Ryan Gosling as good as he is seemed to old to play Ken and Will Ferrell ruined every scene he was in. I just didn't get it, it seemed hollow artificial and hackneyed. A waste of some great talent. It was predictable without being reassuring and

In [ ]:
qa_with_sources_chain({"query" : "Do reviewers consider this movie Kenough?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do reviewers consider this movie Kenough?',
 'result': ' Based on the reviews provided, it seems that the reviewers have mixed opinions about the movie. Some reviewers, such as eoinageary and hamsterination, have positive reviews and rate the movie 8 and 6 out of 10, respectively. However, other reviewers, such as brianjohnson-20043 and holarsch, have more negative reviews and rate the movie 4 and 6 out of 10, respectively. Therefore, it cannot be determined whether the reviewers consider the movie Kenough or not.',
 'source_documents': [Document(page_content=': 37\nReview_Date: 23 July 2023\nAuthor: eoinageary\nRating: 8\nReview_Title: I am Kenough\nReview: So I went into the movie with little to no expectations and I was pleasantly impressed with the movie overall.\nReview_Url: /review/rw9199947/?ref_=tt_urv', metadata={'source': '/review/rw9199947/?ref_=tt_urv', 'row': 37}),
  Document(page_content=": 9\nReview_Date: 19 July 2023\nAuthor: hamsterination\nRating: 6\nReview

In [ ]:
qa_with_sources_chain({"query" : "Did these movies explores themes of existentialism?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Did these movies explores themes of existentialism?',
 'result': ' Based on the reviews, it appears that "Oppenheimer" explores themes of existentialism. The review by sosrivi states that the film is "a reminder to the audience of how obsessed humans are with power and fame," which is a common theme in existentialist literature and film. Additionally, the review by chris_rowe-881-168820 states that the film is "pretty much perfect," which suggests that it effectively explores its themes and ideas.',
 'source_documents': [Document(page_content=": 117\nReview_Date: 20 July 2023\nAuthor: mattlx\nRating: 8\nReview_Title: A decent experience, still not perfect\nReview: The film's abundance of characters, combined with its non-linear narrative approach, might leave you a bit puzzled during your first viewing.\nReview_Url: /review/rw9201962/?ref_=tt_urv", metadata={'source': '/review/rw9201962/?ref_=tt_urv', 'row': 117}),
  Document(page_content=": 135\nReview_Date: 20 July 2023\nA

And with that, we have our Barbie & Oppenheimer Review RAG tool built!

This Notebook is a companion to the event put on by [AIMS](https://www.linkedin.com/company/ai-maker-space/), and [Deci](https://deci.ai/), and is authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)